# IB Status & Issues Follow-UP

This notebook has the objective of creating a Excel file that gather in one unique sheet information for all objects, DCNs and parts being introduced or canceled at VdB factory. Currently these informations are disconected and managed by different systems and files:

- DCNs report from [DIS](http://wwwvtc.volvo.se/dis/): contains information about all the objects and DCNs being introduced in a specific factory in a given period of time. In our case the report contains data for the Brazilian factory, only user 2800 (trucks) for the current, the previous and the next year. **This report contains no information about new or canceled parts for a given object number**;

- [NP-CC](https://volvogroup.sharepoint.com/sites/TP-aecv/ppd/ppd-sa/dcn%20logistics%20flow/forms/allitems.aspx): sheet weekly generated by PO&BO containing information about all the parts to be introduced and cancelled existing in the system in a given week. **This sheet contains no information about objects that do not introduce or cancel new parts in the product structure**.



By creating one unique dataset we will be able to perform statistical analysis and summarize information to support decision making and resources management.

The steps to be followed are:

1. Import and transform both NP-CC sheet and DIS report in unique and separated dataframes;
2. Prepare the two dataframes for the a merge operation;
3. Perform the merge opearation and create one unique dataframe;
3. Clean and prepare the data, create new columns for further categorization;
4. Perform an exploratory data analysis.

### 1) Setting up the enviroment:

In [1]:
import pandas as pd
import numpy as np
import matplotlib
from datetime import date
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('bmh')
pd.set_option('mode.chained_assignment', None)

### 2) Loading the files and creating the data frames:

In [2]:
# creating a dataframe from NP-CC:

np_cc_xlsx = 'NPs_CCs - w2036.1.xlsx'
np_cc = pd.read_excel(np_cc_xlsx)


# creating a dataframe from DIS:

dis2019 = pd.read_excel('2019.xlsx')
dis2020 = pd.read_excel('2020.xlsx')
dis2021 = pd.read_excel('2021.xlsx')
dis = pd.concat([dis2019,dis2020,dis2021])

### 3) Preparing the NP-CC sheet

In [3]:
# changing the column names to make it more 'readable':
np_cc.columns = ['pn', 'object_npcc', 'iw_npcc', 'npcc', 'pn_description', 'dcn_npcc', 'forn_sw',
       'pob', 'ppap_first_plan', 'ppap_approved', 'ppap_y_plan', 'sqa', 'pc_npcc',
       'station', 'mod', 'box']

# converting the pc_npcc column to string type:
np_cc['pc_npcc'] = np_cc['pc_npcc'].apply(lambda x: str(x))

# removing the columns station, mod and box:
np_cc.drop(columns=['station','mod','box'],inplace=True)

# removing all duplicate rows remaining:
np_cc.drop_duplicates(keep='first',inplace=True)

# converting the dcn column to the string format:
np_cc['dcn_npcc'] = np_cc['dcn_npcc'].apply(lambda x: str(x))

# creating NP and CC columns and filling it with 1 or 0:
np_cc['NP'] = np_cc['npcc'].apply(lambda x: 1 if x == "NP" else 0)
np_cc['CC'] = np_cc['npcc'].apply(lambda x: 1 if x == "CC" else 0)

# filling the missing values in the ppap info columns with zero:
np_cc['ppap_first_plan'].fillna(value=0,inplace=True)
np_cc['ppap_y_plan'].fillna(value=0,inplace=True)

# converting the PPAP columns to integer type:
np_cc['ppap_first_plan'] = np_cc['ppap_first_plan'].apply(lambda x: int(x))
np_cc['ppap_y_plan'] = np_cc['ppap_y_plan'].apply(lambda x: int(x))

np_cc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11193 entries, 0 to 11755
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   pn               11193 non-null  object 
 1   object_npcc      10103 non-null  object 
 2   iw_npcc          11193 non-null  int64  
 3   npcc             11193 non-null  object 
 4   pn_description   11192 non-null  object 
 5   dcn_npcc         11193 non-null  object 
 6   forn_sw          11192 non-null  object 
 7   pob              8486 non-null   float64
 8   ppap_first_plan  11193 non-null  int64  
 9   ppap_approved    4618 non-null   object 
 10  ppap_y_plan      11193 non-null  int64  
 11  sqa              6709 non-null   object 
 12  pc_npcc          11193 non-null  object 
 13  NP               11193 non-null  int64  
 14  CC               11193 non-null  int64  
dtypes: float64(1), int64(5), object(9)
memory usage: 1.4+ MB


### 4) Preparing the DIS report

In [4]:
# removing columns that will not be used:
dis.drop(['Cancel Date', 'Appr\nDIS', 'OK\nBer', 'Only\nAD', 'PPC','Final\nUser', 'F\nU'], axis=1, inplace=True)

# renaming the remaining columns
dis.columns = ['dcn_dis', 'dcn_reg_date', 'pc_dis', 'ass_dist', 'as_ta', 'dcn_title', 'object_dis', 'status',
       'handler', 'handler_name', 'iw_dis','rel_user']

# creating a new dcn column as the same format of dcn column in NP-CC sheet:
dis['dcn_npcc'] = dis['dcn_dis'].apply(lambda x: x[1:])

# removing duplicate DCNs:
dis.drop_duplicates(subset = 'dcn_dis', keep='first',inplace=True)

# correcting the pc_dis column: '04 --> 4 and ´06 --> 6:
dis.loc[dis.pc_dis == "'06", 'pc_dis'] = "6"
dis.loc[dis.pc_dis == "'04", 'pc_dis'] = "4"

# creating a function to generate the intro block column
def set_ib(iw_dis): 
    iw_dis = int( str(iw_dis[2:4]) + str(iw_dis[-2:]) )
    rest = iw_dis % 100 # reading the week number
    year = iw_dis - rest # reading the year
    
# creating a conditional since after w2037 the IB changded    
    if iw_dis > 2037: 
    
# if the object is timeset for w4000, it will keep it 
        if iw_dis > 4000: 
            return 4952

        if rest > 5 and rest <= 9: 
            return year + 9
        elif rest > 9 and rest <= 17:
            return year + 17
        elif rest > 17 and rest <= 21:
            return year + 21
        elif rest > 21 and rest <= 27:
            return year + 27
        elif rest > 27 and rest <= 37:
            return year + 37
        elif rest > 37 and rest <= 43:
            return year + 43
        elif rest > 43 and rest <= 47:
            return year + 47
        elif rest > 47:
            return (year+100) + 5
        elif rest <= 5:
            return year + 5
        
    else: 
    
        if rest > 5 and rest <= 11: 
            return year + 11
        elif rest > 11 and rest <= 17:
            return year + 17
        elif rest > 17 and rest <= 23:
            return year + 23
        elif rest > 23 and rest <= 27:
            return year + 27
        elif rest > 27 and rest <= 37:
            return year + 37
        elif rest > 37 and rest <= 43:
            return year + 43
        elif rest > 43 and rest <= 49:
            return year + 49
        elif rest > 49:
            return (year+100) + 5
        elif rest <= 5:
            return year + 5
        
# creating the ib column:
dis['ib_dis'] = dis['iw_dis'].apply(set_ib)

# creating a function to read the year from the ib column we just created: 

def set_ib_year(ib_dis):
    
    ib_str = str(ib_dis)
    two_last = ib_str[:2]
    number_two_last = int(two_last)
    ib_year = 2000 + number_two_last
    return ib_year
    
dis['ib_year'] = dis['ib_dis'].apply(set_ib_year)

# creating a function to define the object category:

def set_object_category(on):

    if on.startswith("12") or on.startswith("52"):
        return "Cost Reduction"
    elif on.startswith("15") or on.startswith("55"):
        return "Market Adaptation"
    elif on.startswith("14") or on.startswith("54"):
        return "Legal Demand"
    elif on.startswith("11") or on.startswith("51"):
        return "Follow UP"
    elif on.startswith("13") or on.startswith("53"):
        return "Quality PMR"
    elif on.startswith("A"):
        return "Son"
    elif on.startswith("Q3") or on.startswith("Q6"):
        return "Protus"
    elif on.startswith("Q"):
        return "QJ"
    elif on.startswith("W60"):
        return "Start Cost"
    else:
        return "Unknown"
    
# applying the function to the object_dis column:

dis['object_category'] = dis['object_dis'].apply(set_object_category)

# converting the iw_dis from string to numeric values, so it can be compared to iw_npcc in NP-CC sheet:
dis['iw_dis'] = dis['iw_dis'].apply(lambda x: int(x[2:4])*100+int(x[-2:]))

### 5) Checking the structure of the two data frames:

In [5]:
dis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2659 entries, 0 to 126
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   dcn_dis          2659 non-null   object        
 1   dcn_reg_date     2659 non-null   datetime64[ns]
 2   pc_dis           2659 non-null   object        
 3   ass_dist         570 non-null    object        
 4   as_ta            2659 non-null   object        
 5   dcn_title        2658 non-null   object        
 6   object_dis       2659 non-null   object        
 7   status           2658 non-null   object        
 8   handler          2659 non-null   object        
 9   handler_name     2659 non-null   object        
 10  iw_dis           2659 non-null   int64         
 11  rel_user         704 non-null    object        
 12  dcn_npcc         2659 non-null   object        
 13  ib_dis           2659 non-null   int64         
 14  ib_year          2659 non-null   int64   

In [6]:
np_cc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11193 entries, 0 to 11755
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   pn               11193 non-null  object 
 1   object_npcc      10103 non-null  object 
 2   iw_npcc          11193 non-null  int64  
 3   npcc             11193 non-null  object 
 4   pn_description   11192 non-null  object 
 5   dcn_npcc         11193 non-null  object 
 6   forn_sw          11192 non-null  object 
 7   pob              8486 non-null   float64
 8   ppap_first_plan  11193 non-null  int64  
 9   ppap_approved    4618 non-null   object 
 10  ppap_y_plan      11193 non-null  int64  
 11  sqa              6709 non-null   object 
 12  pc_npcc          11193 non-null  object 
 13  NP               11193 non-null  int64  
 14  CC               11193 non-null  int64  
dtypes: float64(1), int64(5), object(9)
memory usage: 1.4+ MB


In [7]:
dis.head(3)

,dcn_dis,dcn_reg_date,pc_dis,ass_dist,as_ta,dcn_title,object_dis,status,handler,handler_name,iw_dis,rel_user,dcn_npcc,ib_dis,ib_year,object_category
0,B8519229,2018-07-11,71,NaN,TA,REF DCN FIX,PRO-TEST,NaN,A171500,SENTHIL KUMARAN,1921,GBOX,8519229,1923,2019,Unknown
1,K1290972,2013-10-11,24,81,AS,"CORRECTION OF ASSEMBLY MARKING 81,FASTNER AIR ...",A02-1266,Concluded,A029327,GEOVANNA SEGALIN,1949,NaN,1290972,1949,2019,Son
2,K1332271,2014-12-04,71,NaN,AS,PC71/FG41*&43*/SMT - BOI4159 - CLUTCH SERVO AS...,P32-0108,Concluded,A038759,NICHOLAS ORMENEZE,1933,GBOX,1332271,1937,2019,Unknown


In [8]:
np_cc.head(3)

,pn,object_npcc,iw_npcc,npcc,pn_description,dcn_npcc,forn_sw,pob,ppap_first_plan,ppap_approved,ppap_y_plan,sqa,pc_npcc,NP,CC
0,033023,NaN,1903,NP,CHAS FH 62T RAP6740 CR,22799,41919,100.0,1749,Y,1749,Flavia Toyomoto,24,1,0
1,033024,NaN,1903,NP,CHAS FH 62T RAP6740 CR,22799,41919,100.0,1,NaN,1,Flavia Toyomoto,24,1,0
2,033025,NaN,1903,NP,CHAS FM 62T RAP6740 CR,22799,41919,100.0,1748,Y,1748,Flavia Toyomoto,24,1,0


In [9]:
# checking the dcn_npcc column in the both data frames:

print('DCNs column format for NP-CC sheet:')
print(np_cc['dcn_npcc'].iloc[0::1000])
print('\n')
print('DCNs column format for DIS report:')
print(dis['dcn_npcc'].iloc[0::300])

DCNs column format for NP-CC sheet:
0          22799
1019       22779
2099     1430299
3106       22700
4156       23347
5156     1482625
6265       23647
7436     1500231
8463     1494352
9471     1428258
10477         61
11553    1467174
Name: dcn_npcc, dtype: object


DCNs column format for DIS report:
0       8519229
301     1430121
602     1446253
906     1455550
1206    1463948
1506    1472091
1809    1481516
160     1482906
462     1500557
Name: dcn_npcc, dtype: object


### 6) Merging the data frames:

The _outer merge_ operation will add the columns of NP-CC data frame on the DIS report data frame. For the rows in the NP-CC data frame that have no corresponding instances in the DIS report data frame, the columns from DIS report data frame will be kept empty.

In [10]:
# the command below performs the merging operation:
disnpcc = pd.merge(left=dis, right=np_cc, left_on='dcn_npcc', right_on='dcn_npcc', how='outer')
disnpcc.head(3)

# dropping the empty values from object_dis column:
disnpcc = disnpcc.dropna(subset=['object_dis'])
disnpcc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5141 entries, 0 to 5140
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   dcn_dis          5141 non-null   object        
 1   dcn_reg_date     5141 non-null   datetime64[ns]
 2   pc_dis           5141 non-null   object        
 3   ass_dist         939 non-null    object        
 4   as_ta            5141 non-null   object        
 5   dcn_title        5140 non-null   object        
 6   object_dis       5141 non-null   object        
 7   status           5140 non-null   object        
 8   handler          5141 non-null   object        
 9   handler_name     5141 non-null   object        
 10  iw_dis           5141 non-null   float64       
 11  rel_user         1032 non-null   object        
 12  dcn_npcc         5141 non-null   object        
 13  ib_dis           5141 non-null   float64       
 14  ib_year          5141 non-null   float64

The merging operation is complete. We are now able to generate an excel file to work with. But before, let's clean some columns and reorganize the column order.

In [12]:
# creating the merged data frame version to be converted to excel:

disnpcc_report = disnpcc[['object_dis','object_category','dcn_dis','dcn_title','iw_dis','iw_npcc','ib_dis',
                          'pn','pn_description','pc_npcc','npcc','NP','CC','pob','ppap_first_plan', 'ppap_approved', 
                          'ppap_y_plan', 'sqa']]

# we will now proceed to the creation of the excel file, with the following format: "disnpcc_report_YYYY.WW.D.xlsx":

year = date.today().strftime("%Y")
week_str = date.today().strftime("%U")
week = int(week_str)+1
day = date.today().strftime("%w")

disnpcc_report.to_excel('disnpcc_report_{0}.{1}.{2}.xlsx'.format(year,week,day))

The excel file is now generated for daily consulting and follow-up. We can now proceed to EDA and creation of filters to track issues.

### 7) Exploratory Data Analysis and Issues flitering: